# Shor's Algorithm Implementation

## Importing Libraries

In [ ]:
"""Install Cirq."""

try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")

installing cirq...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 68.5 MB/s eta 0:00:00
installed cirq.


In [ ]:
"""Imports for the notebook."""

import fractions
import math
import random

import numpy as np
import sympy
from typing import Callable, Iterable, Sequence

import cirq

## Order Finding


### 1. The big picture — what “Order Finding” is about

The **goal** is to factor a large number $n$ (find its prime factors $p$ and $q$).

Shor's algorithm (in quantum computing) does this by turning **factoring** into a simpler problem called **order finding**.

So, instead of trying to directly find $p$ and $q$, we find something called the **order** of a number modulo $n$.

## 2. What does “modulo” mean again?

When we say
$$
a \mod n
$$
we mean **the remainder when $a$ is divided by $n$**.

Example:

* $10 \mod 3 = 1$ because $10 = 3×3 + 1$
* $7 \mod 5 = 2$

So when we do arithmetic “mod n”, we **wrap around** at $n$.
It's like a clock with $n$ hours — when you reach $n$, you go back to 0.

## 3. What is $\mathbb{Z}_n$?

$$
\mathbb{Z}_n = { x \in {1, 2, \dots, n-1} \mid \gcd(x, n) = 1 }
$$

That means:

* It's the set of numbers less than $n$ that are **coprime** to $n$ (they share no common divisor other than 1).
* “Coprime” = their greatest common divisor (gcd) with $n$ is 1.

We call this set **the multiplicative group modulo n**, because:

* We can multiply elements together, and
* Every element has a *multiplicative inverse* (i.e., something you can multiply by to get 1 mod n).

## 4. Example: $n = 15$

Let's find all numbers less than 15 that are coprime to it.

$$
\text{Numbers < 15: } 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14
$$

Now remove those that share factors with 15 (which is $3 \times 5$):

* Remove all multiples of 3 → (3, 6, 9, 12)
* Remove all multiples of 5 → (5, 10)

Remaining numbers:
$$
\mathbb{Z}_{15} = {1, 2, 4, 7, 8, 11, 13, 14}
$$

These 8 numbers form a **group** under multiplication mod 15.

## 5. What is the “order” of an element?

If you take a number $x$ in $\mathbb{Z}_n$, multiply it by itself repeatedly (mod n), at some point the result becomes 1 again.

That smallest positive exponent $r$ that makes:
$$
x^r \mod n = 1
$$
is called the **order** of $x$ modulo $n$.

### Example: Find the order of $x = 2$ mod $15$

Compute:

| Power | $2^k \mod 15$ | Result |
| ----- | --------------- | ------ |
| 1     | 2               | 2      |
| 2     | 4               | 4      |
| 3     | 8               | 8      |
| 4     | 16 mod 15 = 1   | ✅      |

So $2^4 \mod 15 = 1$

The **order** of 2 mod 15 is **r = 4**.

## 6. The modular exponential function

We define a function:
$$
f_x(z) = x^z \mod n
$$

This function **repeats every r steps**, because when you go past the order, the results start cycling again:

$$
f_x(z + r) = f_x(z)
$$

That's what we mean when we say **it's periodic with period $r$**.
### Example continued:

For $x = 2, n = 15$:

| z | $f_2(z) = 2^z \mod 15$ |
| - | ------------------------ |
| 0 | 1                        |
| 1 | 2                        |
| 2 | 4                        |
| 3 | 8                        |
| 4 | 1                        |
| 5 | 2                        |
| 6 | 4                        |
| 7 | 8                        |
| 8 | 1                        |

See how it repeats every 4 steps?
So **period = 4**.

## 7. Why is this important for factoring?

Now here's the clever trick (used by Shor's algorithm):

If $n = pq$ (product of two primes), and we randomly pick some $x$ that's coprime to $n$, then find its order $r$:

* If $r$ is **even**, and
  $x^{r/2} \not\equiv -1 \pmod{n}$,

then we can compute:
$$
\gcd(x^{r/2} - 1, n) \quad \text{and} \quad \gcd(x^{r/2} + 1, n)
$$
and these two gcds will (with high probability) give us **p and q**, the prime factors of $n$.

So — if we can **find the order r**, we can **factor n**.

## 8. Why quantum computers care

On a classical computer, finding the order $r$ takes too long for big $n$.
But on a **quantum computer**, using the **Quantum Fourier Transform (QFT)**, we can find $r$ efficiently.

That's why **Shor's algorithm** breaks RSA — because RSA's security relies on factoring being hard.

In [ ]:
"""Function to compute the elements of Z_n."""


def multiplicative_group(n: int) -> list[int]:
    """Returns the multiplicative group modulo n.

    Args:
        n: Modulus of the multiplicative group.
    """
    assert n > 1
    group = [1]
    for x in range(2, n):
        if math.gcd(x, n) == 1:
            group.append(x)
    return group

For example, the multiplicative group modulo $n = 15$ is shown below.


In [ ]:
"""Example of a multiplicative group."""

n = 15
print(f"The multiplicative group modulo n = {n} is:")
print(multiplicative_group(n))

The multiplicative group modulo n = 15 is:
[1, 2, 4, 7, 8, 11, 13, 14]


## Classical Order Finding

So far we learned what **order** means —
it's the smallest positive integer $r$ such that:

$$
x^r \mod n = 1
$$

Now we'll see **how to find this order using a normal (classical) computer** — and why it's *slow* compared to the quantum version.


### The idea

The classical algorithm is basically **trial and error**:

1. Start with $x$
2. Compute $x^2 \mod n$
3. Compute $x^3 \mod n$
4. Keep going: $x^4 \mod n$, $x^5 \mod n$, …
5. Stop when you reach **1** again — that exponent is $r$.

That's literally all it does!

### Example: $n = 15, x = 8$

Let's compute step by step:

| Power $k$ | $8^k \mod 15$          | Result |
| ----------- | ------------------------ | ------ |
| 1           | 8                        | 8      |
| 2           | $64 \mod 15 = 4$       | 4      |
| 3           | $8×4 = 32 \mod 15 = 2$ | 2      |
| 4           | $8×2 = 16 \mod 15 = 1$ |  1    |

So we reached 1 again at $k = 4$.
Therefore, **the order of 8 mod 15 is r = 4.**

### Why is this inefficient?

Because in the **worst case**, we might have to check **all possible exponents** up to the size of the group $|\mathbb{Z}_n| = \phi(n)$.

* $\phi(n)$ (Euler's totient function) is roughly the number of integers less than $n$ that are coprime to it.
* For $n = pq$ (two primes), $\phi(n) = (p - 1)(q - 1)$.

So, in the worst case, the number of steps ≈ $\phi(n)$.

That's why the **time complexity** is:

$$
O(\phi(n)) \approx O(2^{L/2})
$$

where $L$ = number of bits in $n$.

In other words:

* If $n$ has 2048 bits (typical RSA size),
  this means ~$2^{1024}$ operations → **impossible** to compute in practice.

### Why we need quantum help

The **quantum part of Shor's algorithm** replaces this slow, step-by-step checking with a **quantum parallel** approach that can find the order **exponentially faster**.

Classically → $O(2^{L/2})$
Quantumly → **Polynomial time** (roughly $O(L^3)$)

That's the revolutionary step:
The quantum circuit for **period finding** (i.e., finding $r$) makes factoring fast.

In [ ]:
"""Function for classically computing the order of an element of Z_n."""


def classical_order_finder(x: int, n: int) -> int | None:
    """Computes smallest positive r such that x**r mod n == 1.

    Args:
        x: Integer whose order is to be computed, must be greater than one
           and belong to the multiplicative group of integers modulo n (which
           consists of positive integers relatively prime to n),
        n: Modulus of the multiplicative group.

    Returns:
        Smallest positive integer r such that x**r == 1 mod n.
        Always succeeds (and hence never returns None).

    Raises:
        ValueError when x is 1 or not an element of the multiplicative
        group of integers modulo n.
    """
    # Make sure x is both valid and in Z_n.
    if x < 2 or x >= n or math.gcd(x, n) > 1:
        raise ValueError(f"Invalid x={x} for modulus n={n}.")

    # Determine the order.
    r, y = 1, x
    while y != 1:
        y = (x * y) % n
        r += 1
    return r

An example of computing $r$ for a given $x \in \mathbb{Z}_n$ and given $n$ is shown in the code block below.

In [ ]:
"""Example of (classically) computing the order of an element."""

n = 15  # The multiplicative group is [1, 2, 4, 7, 8, 11, 13, 14].
x = 8
r = classical_order_finder(x, n)

# Check that the order is indeed correct.
print(f"x^r mod n = {x}^{r} mod {n} = {x**r % n}")

x^r mod n = 8^4 mod 15 = 1


## Quantum order finding

Quantum order finding is essentially quantum phase estimation with unitary $U$ that computes the modular exponential function $f_x(z)$ for some randomly chosen $x \in \mathbb{Z}_n$. The full details of how $U$ is computed in terms of elementary gates can be complex to unravel, especially on a first reading. In this tutorial, we'll use arithmetic gates in Cirq which can implement such a unitary $U$ without fully delving into the details of elementary gates.

Below we first show an example of a simple arithmetic gate in Cirq (addition) then discuss the gate we care about (modular exponentiation).

### Quantum Arithmetic Gates in Cirq

### What's the idea?

In classical computing, addition is easy:
$$
(a, b) \mapsto (a, a + b)
$$
You just add the numbers.

In **quantum computing**, we need operations that are:

* **Reversible**, and
* **Act on qubits**, not integers directly.

That's where **quantum arithmetic gates** come in.
They let us perform **mathematical operations (like add, multiply, mod, etc.)** directly on **quantum registers** (groups of qubits).


## What does this Adder gate do?

This special gate called **`Adder`** takes two quantum registers:

* The **input register** (denoted $i$): holds $a$
* The **target register** (denoted $t$): holds $b$

It performs the operation:

$$
|a\rangle_i |b\rangle_t \longmapsto |a\rangle_i |(a + b) \bmod N_t\rangle_t
$$

So, it adds the value in the **input** register to the **target** register, but keeps the input register unchanged.

- The **target register** gets updated
- The **input register** is left as is

Because quantum gates must be **reversible**, the addition is done **modulo** the size of the register ($N_t$), so you can always “undo” it later.

## The math meaning (in plain words)

Let's imagine both registers are 2 qubits each.
Each register can represent values 0, 1, 2, or 3 (since (2^2 = 4)).

Example:

| Register | Binary      | Decimal |
| -------- | ----------- | ------- |
| $10\rangle$ |  10  |  2     |
| $01\rangle$ |  01  |   1    |

Then after applying `Adder`:

$$
|10\rangle_i |01\rangle_t \longrightarrow |10\rangle_i |11\rangle_t
$$

because ( 2 + 1 = 3 ) (in binary: ( 10 + 01 = 11 )).

- Input (10) unchanged
- Target (01 → 11) updated

## How Cirq handles this

When defining **arithmetic gates**, we can subclass `cirq.ArithmeticGate`.
This lets us describe operations using **integer arithmetic**, and Cirq handles all the quantum bit-level work behind the scenes.

### Code Breakdown

```python
class Adder(cirq.ArithmeticGate):
    """Quantum addition."""

    def __init__(self, target_register, input_register):
        self.target_register = target_register
        self.input_register = input_register
```

Here we define which qubits belong to which register.

#### `registers()`

This tells Cirq the **structure** of the registers (which qubits belong where).

```python
def registers(self):
    return self.target_register, self.input_register
```

#### `apply()`

This is where we tell Cirq what mathematical operation to perform.

```python
def apply(self, *register_values):
    return sum(register_values)
```

The sum is interpreted as **addition mod N**, where $N$ is automatically the size of the target register.
That's why we do'’t explicitly write “mod N” — Cirq infers it automatically.

#### `_circuit_diagram_info_()`

Just defines how it should appear in a circuit diagram (“+” symbol).

In [ ]:
"""Example of defining an arithmetic (quantum) gate in Cirq."""


class Adder(cirq.ArithmeticGate):
    """Quantum addition."""

    def __init__(self, target_register: int | Sequence[int], input_register: int | Sequence[int]):
        self.target_register = target_register
        self.input_register = input_register

    def registers(self) -> Sequence[int | Sequence[int]]:
        return self.target_register, self.input_register

    def with_registers(self, *new_registers: int | Sequence[int]) -> 'Adder':
        return Adder(*new_registers)

    def apply(self, *register_values: int) -> int | Iterable[int]:
        return sum(register_values)

    def _circuit_diagram_info_(self, args: cirq.CircuitDiagramInfoArgs):
        wire_symbols = [' + ' for _ in range(len(self.input_register) + len(self.target_register))]
        return cirq.CircuitDiagramInfo(wire_symbols=tuple(wire_symbols))

## Building and Using the Circuit

Now that we've defined the `Adder` gate, we can **test** it on an example circuit.

### Step 1: Create two registers

In [ ]:
qreg1 = cirq.LineQubit.range(2)  # input
qreg2 = cirq.LineQubit.range(2, 4)  # target

### Step 2: Initialize them

We use **X gates** to flip certain qubits from 0 to 1.



In [ ]:
cirq.X.on(qreg1[0])  # sets input register to |10> (binary 2)
cirq.X.on(qreg2[1])  # sets target register to |01> (binary 1)

cirq.X(cirq.LineQubit(3))

So before addition:
$$
|a\rangle_i |b\rangle_t = |10\rangle_i |01\rangle_t
$$
### Step 3: Apply the Adder gate

In [ ]:
adder = Adder(input_register=[2, 2], target_register=[2, 2])
circ = cirq.Circuit(
    cirq.X.on(qreg1[0]),
    cirq.X.on(qreg2[1]),
    adder.on(*qreg1, *qreg2),
    cirq.measure_each(*qreg1),
    cirq.measure_each(*qreg2),
)

# Display it.
print("Circuit:\n")
print(circ)

Circuit:

0: ───X─── + ───M───
          │
1: ─────── + ───M───
          │
2: ─────── + ───M───
          │
3: ───X─── + ───M───


Then we **measure** both registers to see the result.

### Step 4: Expected output

Because we added 2 + 1 = 3:

* Input register = $|10\rangle$ (unchanged)
* Target register = $|11\rangle$

Combined bitstring = **`1011`**

So when we simulate multiple times:


In [ ]:
# Print the measurement outcomes.
print("\n\nMeasurement outcomes:\n")
print(cirq.sample(circ, repetitions=5).data)



Measurement outcomes:

   q(0)  q(1)  q(2)  q(3)
0     1     1     0     1
1     1     1     0     1
2     1     1     0     1
3     1     1     0     1
4     1     1     0     1


we’ll always get:

```
1011
1011
1011
1011
1011
```

Perfect match with theory!

## Viewing the Unitary Matrix

We can also look at the **unitary matrix** (the mathematical representation of the gate).



In [ ]:
cirq.unitary(Adder(target_register=[2, 2], input_register=1)).real

array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])

This shows how each possible input state is transformed.
Each **column** represents what happens to a particular input basis state.

### Intuitive meaning of the unitary:

The ( i^{th} ) column of the unitary corresponds to the state:

$$
|i + 1 \bmod 4\rangle
$$

For 2 qubits (4 possible values):

| i | Operation   | Resulting state |    |
| - | ----------- | --------------- | -- |
| 0 | 0 + 1 mod 4 |                 | 1⟩ |
| 1 | 1 + 1 mod 4 |                 | 2⟩ |
| 2 | 2 + 1 mod 4 |                 | 3⟩ |
| 3 | 3 + 1 mod 4 |                 | 0⟩ |

So the unitary basically **shifts** all states by +1 (mod 4) — exactly what modular addition does!

## Modular Exponentiation in Shor's Algorithm

The **goal** of the modular exponentiation unitary $U$ in Shor's algorithm is to efficiently perform the transformation:

$$
U |y⟩ = |x \cdot y \bmod n⟩
$$

and, by extension, to enable computing powers of $U$:

$$
U^e |y⟩ = |x^e \cdot y \bmod n⟩
$$

This operation forms the **core quantum subroutine** used in **phase estimation**, which ultimately extracts the order $r$ of $x$ modulo $n$.


### Why Modular Exponentiation?

Recall:
In the **classical order finding** function, we computed $x^r \bmod n$ step by step, which takes time **O(φ(n))**, i.e., exponential in the bit size of $n$.

Quantumly, we want to do the same — find ( r ) — but **much faster**, using **quantum parallelism**.
That's why we need a **reversible quantum circuit** for modular exponentiation.

### The `ModularExp` Gate — Conceptual Overview

The provided class:

```python
class ModularExp(cirq.ArithmeticGate):
```

inherits from Cirq's `ArithmeticGate` to create a **custom gate** that performs modular exponentiation.

Its main idea is to apply the transformation:

$$
|y⟩|e⟩ \mapsto |y \cdot x^e \bmod n⟩ |e⟩
$$

so that the **second register (`exponent`) controls the exponentiation**, while the **first register (`target`) is multiplied accordingly**.

### Key Components in the Class

#### 1. **Initialization**

```python
def __init__(self, target, exponent, base, modulus):
```

* `target`: quantum register for $y$
* `exponent`: quantum register for $e$
* `base`: the classical constant $x \in \mathbb{Z}_n$
* `modulus`: the integer $n$ to factor

The constructor ensures the target register is large enough to represent numbers up to $n$.

#### 2. **The `apply` Method**

This is the mathematical definition of the unitary.
It implements the modular multiplication:

```python
return (target * base**exponent) % modulus
```

* The operation is performed only if `target < modulus`;
  otherwise, it returns the target unchanged (to preserve reversibility outside valid range).

This line is the heart of the modular exponentiation in Shor’s algorithm — the **quantum version** of $y \mapsto y \cdot x^e \bmod n$.

#### 3. **Diagram Info**

The `_circuit_diagram_info_` method simply tells Cirq **how to label the wires** in a quantum circuit visualization (e.g., `t0`, `t1`, `e0`, etc.).

### Register Sizes and Qubit Count

For an integer $n$:

* $L = \text{bit length of } n$
* Target register: $L$ qubits
* Exponent register: $2L + 3$ qubits
* Total qubits: $3L + 3$

Hence, for $n = 15$ (4 bits),
we need $3(4) + 3 = 15$ qubits.

```python
n = 15
L = n.bit_length()  # 4 bits
target = cirq.LineQubit.range(L)
exponent = cirq.LineQubit.range(L, 3 * L + 3)
print(f"To factor n = {n} which has L = {L} bits, we need 3L + 3 = {3 * L + 3} qubits.")
```

Output:

```
To factor n = 15 which has L = 4 bits, we need 3L + 3 = 15 qubits.
```

### Memory Constraints (Important)

The full unitary matrix for modular exponentiation is **massive**:

$$
\text{Size} = 4^{3(L+1)}
$$

Even for small $n = 15$, this means storing $2^{30}$ floating-point numbers — roughly **1 GB of RAM** just for this unitary, which is impractical to visualize or simulate fully on a laptop.

That's why in real simulations, we don't actually build this matrix — we use **structured gates** or **optimized routines**.

---

### Example of Using the Gate

```python
x = 5  # Example base in Z_n
# cirq.unitary(ModularExp(target, exponent, x, n))  # Uncomment for small n
```

This would (if memory allowed) display the **unitary matrix** for the modular exponentiation operator that applies $|y⟩|e⟩ \mapsto |y * 5^e \bmod 15⟩|e⟩$.

In [ ]:
"""Defines the modular exponential gate used in Shor's algorithm."""


class ModularExp(cirq.ArithmeticGate):
    """Quantum modular exponentiation.

    This class represents the unitary which multiplies base raised to exponent
    into the target modulo the given modulus. More precisely, it represents the
    unitary V which computes modular exponentiation x**e mod n:

        V|y⟩|e⟩ = |y * x**e mod n⟩ |e⟩     0 <= y < n
        V|y⟩|e⟩ = |y⟩ |e⟩                  n <= y

    where y is the target register, e is the exponent register, x is the base
    and n is the modulus. Consequently,

        V|y⟩|e⟩ = (U**e|y)|e⟩

    where U is the unitary defined as

        U|y⟩ = |y * x mod n⟩      0 <= y < n
        U|y⟩ = |y⟩                n <= y
    """

    def __init__(
        self, target: Sequence[int], exponent: int | Sequence[int], base: int, modulus: int
    ) -> None:
        if len(target) < modulus.bit_length():
            raise ValueError(
                f'Register with {len(target)} qubits is too small for modulus' f' {modulus}'
            )
        self.target = target
        self.exponent = exponent
        self.base = base
        self.modulus = modulus

    def registers(self) -> Sequence[int | Sequence[int]]:
        return self.target, self.exponent, self.base, self.modulus

    def with_registers(self, *new_registers: int | Sequence[int]) -> 'ModularExp':
        """Returns a new ModularExp object with new registers."""
        if len(new_registers) != 4:
            raise ValueError(
                f'Expected 4 registers (target, exponent, base, '
                f'modulus), but got {len(new_registers)}'
            )
        target, exponent, base, modulus = new_registers
        if not isinstance(target, Sequence):
            raise ValueError(f'Target must be a qubit register, got {type(target)}')
        if not isinstance(base, int):
            raise ValueError(f'Base must be a classical constant, got {type(base)}')
        if not isinstance(modulus, int):
            raise ValueError(f'Modulus must be a classical constant, got {type(modulus)}')
        return ModularExp(target, exponent, base, modulus)

    def apply(self, *register_values: int) -> int:
        """Applies modular exponentiation to the registers.

        Four values should be passed in.  They are, in order:
          - the target
          - the exponent
          - the base
          - the modulus

        Note that the target and exponent should be qubit
        registers, while the base and modulus should be
        constant parameters that control the resulting unitary.
        """
        assert len(register_values) == 4
        target, exponent, base, modulus = register_values
        if target >= modulus:
            return target
        return (target * base**exponent) % modulus

    def _circuit_diagram_info_(self, args: cirq.CircuitDiagramInfoArgs) -> cirq.CircuitDiagramInfo:
        """Returns a 'CircuitDiagramInfo' object for printing circuits.

        This function just returns information on how to print this operation
        out in a circuit diagram so that the registers are labeled
        appropriately as exponent ('e') and target ('t').
        """
        assert args.known_qubits is not None
        wire_symbols = [f't{i}' for i in range(len(self.target))]
        e_str = str(self.exponent)
        if isinstance(self.exponent, Sequence):
            e_str = 'e'
            wire_symbols += [f'e{i}' for i in range(len(self.exponent))]
        wire_symbols[0] = f'ModularExp(t*{self.base}**{e_str} % {self.modulus})'
        return cirq.CircuitDiagramInfo(wire_symbols=tuple(wire_symbols))

In [ ]:
"""Create the target and exponent registers for phase estimation,
and see the number of qubits needed for Shor's algorithm.
"""

n = 15
L = n.bit_length()

# The target register has L qubits.
target = cirq.LineQubit.range(L)

# The exponent register has 2L + 3 qubits.
exponent = cirq.LineQubit.range(L, 3 * L + 3)

# Display the total number of qubits to factor this n.
print(f"To factor n = {n} which has L = {L} bits, we need 3L + 3 = {3 * L + 3} qubits.")

To factor n = 15 which has L = 4 bits, we need 3L + 3 = 15 qubits.


## Quantum Order Finding via Phase Estimation

The quantum core of Shor's algorithm is **phase estimation** using the **modular exponential unitary** $U$, defined as:

$$
U|y⟩ = |x \cdot y \bmod n⟩
$$

The goal is to estimate the **phase** (or eigenvalue) of $U$, which encodes information about the **order $r$** — the smallest positive integer such that:

$$
x^r \equiv 1 \pmod{n}
$$

Once we estimate this phase quantumly, we can **classically compute $r$** and thus factor $n$.

## The `make_order_finding_circuit()` Function Explained

This function constructs the **quantum circuit** implementing **phase estimation** using the `ModularExp` gate.

Let's break it down carefully:

### 1. Define Registers

```python
L = n.bit_length()
target = cirq.LineQubit.range(L)
exponent = cirq.LineQubit.range(L, 3 * L + 3)
```

* `target`: register of **L qubits**, holds values $|y⟩$
* `exponent`: register of **2L + 3 qubits**, controls powers of $U$ (the exponent part)

The total qubit count is $3L + 3$, as before.

### 2. Define the Modular Exponentiation Gate

```python
mod_exp = ModularExp([2] * L, [2] * (2 * L + 3), x, n)
```

This creates a **custom arithmetic gate** that performs modular exponentiation:
$$
|y⟩|e⟩ \mapsto |y \cdot x^e \bmod n⟩ |e⟩
$$

where $x$ is the base and $n$ is the modulus.

### 3. Build the Quantum Circuit

```python
return cirq.Circuit(
    cirq.X(target[L - 1]),
    cirq.H.on_each(*exponent),
    mod_exp.on(*target, *exponent),
    cirq.qft(*exponent, inverse=True),
    cirq.measure(*exponent, key='exponent'),
)
```

Let's explain each step

#### Step 1: `cirq.X(target[L - 1])`

* Puts the **target register** into the state (|1⟩) instead of (|0⟩).
* This is necessary for **phase kickback** — the mechanism by which the phase (related to $r$) is transferred to the exponent register.

#### Step 2: `cirq.H.on_each(*exponent)`

* Applies **Hadamard gates** to all qubits in the **exponent register**.
* This creates an **equal superposition** over all possible exponents:
  $$
  \frac{1}{\sqrt{2^t}} \sum_{e=0}^{2^t-1} |e⟩
  $$
  where $t = 2L + 3$.

#### Step 3: `mod_exp.on(*target, *exponent)`

* Applies the **controlled modular exponentiation**.
* This effectively performs:
  $$
  |y⟩|e⟩ \mapsto |y \cdot x^e \bmod n⟩ |e⟩
  $$
* This is where **phase information** encoding the **order $r$** gets imprinted in the quantum state.

#### Step 4: `cirq.qft(*exponent, inverse=True)`

* Applies the **inverse Quantum Fourier Transform** (QFT⁻¹) to the exponent register.
* This step extracts the **phase** encoded by the modular exponentiation (similar to a discrete Fourier transform detecting periodicity).


#### Step 5: `cirq.measure(*exponent, key='exponent')`

* Measures the **exponent register**, collapsing the superposition into a value $c$.
* This measured value is related to the fraction $s/r$, where $r$ is the period (the order we seek).


### Example — Building the Circuit

```python
n = 15
x = 7
circuit = make_order_finding_circuit(x, n)
print(circuit)
```

This prints a circuit that looks roughly like:

```
0: ────────────────X───────────────────────────────@───────────────────────────────
                                                    │
1: ────────────────────────────────────────────────@───────────────────────────────
      ...
10: ────────────────H──────────────────────────────────────────────────────────────
11: ────────────────H──────────────────────────────────────────────────────────────
        ...
(then inverse QFT and measurement)
```

Each `H` is part of the superposition, the `@` gates belong to modular exponentiation, and the final `QFT⁻¹` and measurement extract the periodicity.


## Sampling the Circuit — Example

Now we can **simulate** the circuit to observe measurement outcomes.

```python
circuit = make_order_finding_circuit(x=5, n=6)
res = cirq.sample(circuit, repetitions=8)

print("Raw measurements:")
print(res)

print("\nInteger in exponent register:")
print(res.data)
```

Notes:

* Here $n = 6$ (an even number) is **just for demonstration** — normally we only factor **odd composite numbers**.
* Each measurement gives a **bitstring** for the exponent register.
* These bitstrings represent integers corresponding to **fractions of $s/r$**, from which $r$ can be deduced.

## What the Measurements Mean

Each measurement result corresponds to an estimate of the **phase** $\phi = s / r$, where $s$ and $r$ are integers and $r$ is the **order**.

Quantum Phase Estimation (QPE) gives values close to $s / r$.
From these, we can recover $r$ **classically** using the **continued fractions algorithm** — that's what the **next section** will cover.


In [ ]:
"""Function to make the quantum circuit for order finding."""


def make_order_finding_circuit(x: int, n: int) -> cirq.Circuit:
    """Returns quantum circuit which computes the order of x modulo n.

    The circuit uses Quantum Phase Estimation to compute an eigenvalue of
    the following unitary:

        U|y⟩ = |y * x mod n⟩      0 <= y < n
        U|y⟩ = |y⟩                n <= y

    Args:
        x: positive integer whose order modulo n is to be found
        n: modulus relative to which the order of x is to be found

    Returns:
        Quantum circuit for finding the order of x modulo n
    """
    L = n.bit_length()
    target = cirq.LineQubit.range(L)
    exponent = cirq.LineQubit.range(L, 3 * L + 3)

    # Create a ModularExp gate sized for these registers.
    mod_exp = ModularExp([2] * L, [2] * (2 * L + 3), x, n)

    return cirq.Circuit(
        cirq.X(target[L - 1]),
        cirq.H.on_each(*exponent),
        mod_exp.on(*target, *exponent),
        cirq.qft(*exponent, inverse=True),
        cirq.measure(*exponent, key='exponent'),
    )

In [ ]:
"""Example of the quantum circuit for period finding."""

n = 15
x = 7
circuit = make_order_finding_circuit(x, n)
print(circuit)

0: ────────ModularExp(t*7**e % 15)────────────────────────────
           │
1: ────────t1─────────────────────────────────────────────────
           │
2: ────────t2─────────────────────────────────────────────────
           │
3: ────X───t3─────────────────────────────────────────────────
           │
4: ────H───e0────────────────────────qft^-1───M('exponent')───
           │                         │        │
5: ────H───e1────────────────────────#2───────M───────────────
           │                         │        │
6: ────H───e2────────────────────────#3───────M───────────────
           │                         │        │
7: ────H───e3────────────────────────#4───────M───────────────
           │                         │        │
8: ────H───e4────────────────────────#5───────M───────────────
           │                         │        │
9: ────H───e5────────────────────────#6───────M───────────────
           │                         │        │
10: ───H───e6─────────────────

In [ ]:
"""Measuring Shor's period finding circuit."""

circuit = make_order_finding_circuit(x=5, n=6)
res = cirq.sample(circuit, repetitions=8)

print("Raw measurements:")
print(res)

print("\nInteger in exponent register:")
print(res.data)

Raw measurements:
exponent=10000000, 00000000, 00000000, 00000000, 00000000, 00000000, 00000000, 00000000, 00000000

Integer in exponent register:
   exponent
0       256
1         0
2         0
3         0
4         0
5         0
6         0
7         0


## Classical Post-Processing

After running the **quantum phase estimation (QPE)** part of Shor's algorithm, we measure an integer value from the **exponent register**.
That measurement encodes a phase approximately equal to:

$$
\phi = \frac{s}{r}
$$

where:

* $r$ = the **order** we want to find,
* $s$ = some integer $0 \le s < r$,
* $\phi$ = the **eigenphase** of the modular exponentiation operator $U$.

The classical post-processing step converts this measured eigenphase into the integer $r$.

### Function: `process_measurement(result, x, n)`

Let's go line by line through this function:

```python
def process_measurement(result: cirq.Result, x: int, n: int) -> int | None:
```

This function takes the **measurement result** of the quantum circuit and tries to recover the **order** $r$ of $x$ modulo $n$.

### 1. Extract the Measured Integer

```python
exponent_as_integer = result.data["exponent"][0]
exponent_num_bits = result.measurements["exponent"].shape[1]
```

* The measurement result of the exponent register is a bitstring, e.g. `10110`.
* Cirq stores it as a pandas DataFrame, so we extract the **integer value** represented by that bitstring.
* `exponent_num_bits` gives the **number of qubits** used in that register, i.e. the denominator size for the phase fraction.

### 2. Compute the Eigenphase

```python
eigenphase = float(exponent_as_integer / 2**exponent_num_bits)
```

Since phase estimation encodes eigenvalues as $e^{2\pi i \phi}$,
the measured integer gives an approximation to $\phi$ as:

$$
\phi = \frac{s}{r} \approx \frac{\text{measured integer}}{2^{t}}
$$

where $t$ is the number of exponent qubits.

### 3. Use Continued Fractions to Approximate $s/r$

```python
f = fractions.Fraction.from_float(eigenphase).limit_denominator(n)
```

This line uses **Python's `fractions` module** to approximate the floating-point number $\phi$ by a rational fraction $s / r$.

We set the maximum denominator to $n$, since the true order $r$ cannot exceed $n$.

### 4. Check Validity

```python
if f.numerator == 0:
    return None
```

If $s = 0$, the measurement didn't give us useful phase information — the algorithm failed this run.

### 5. Validate the Candidate Order $r$

```python
r = f.denominator
if x**r % n != 1:
    return None
```

We verify whether the candidate $r$ actually satisfies the **order condition**:

$$
x^r \equiv 1 \pmod{n}
$$

If it doesn't, it's an incorrect result (can happen due to probabilistic nature of QPE), and we return `None`.

Otherwise, $r$ is our **successfully found order**.


### Full Function Recap

```python
def process_measurement(result: cirq.Result, x: int, n: int) -> int | None:
    exponent_as_integer = result.data["exponent"][0]
    exponent_num_bits = result.measurements["exponent"].shape[1]
    eigenphase = float(exponent_as_integer / 2**exponent_num_bits)

    f = fractions.Fraction.from_float(eigenphase).limit_denominator(n)

    if f.numerator == 0:
        return None

    r = f.denominator
    if x**r % n != 1:
        return None
    return r
```

---

## 🧪 Example Run

```python
# Set n and x
n = 6
x = 5

print(f"Finding the order of x = {x} modulo n = {n}\n")

# Run the quantum order-finding circuit
circuit = make_order_finding_circuit(x, n)
measurement = cirq.sample(circuit, repetitions=1)

print("Raw measurements:")
print(measurement)

print("\nInteger in exponent register:")
print(measurement.data)

r = process_measurement(measurement, x, n)
print("\nOrder r =", r)
if r is not None:
    print(f"x^r mod n = {x}^{r} mod {n} = {x**r % n}")
```

### Expected Output & Reasoning

We should get:

```
Order r = 2
x^r mod n = 5^2 mod 6 = 1
```

Explanation:

* For $n = 6$, $x = 5$:
  $$
  5^1 \bmod 6 = 5, \quad 5^2 \bmod 6 = 25 \bmod 6 = 1
  $$
  Thus, the order $r = 2$.

If the algorithm returns `None`, re-run it — quantum measurement outcomes are probabilistic.

## What Happens Next?

Once we've found $r$, we can **factor $n$** using the classical post-processing of Shor's algorithm:

$$
\text{If } r \text{ is even, compute } \gcd(x^{r/2} \pm 1, n)
$$

This step reveals **non-trivial factors** of ( n ).

In [ ]:
def process_measurement(result: cirq.Result, x: int, n: int) -> int | None:
    """Interprets the output of the order finding circuit.

    Specifically, it determines s/r such that exp(2πis/r) is an eigenvalue
    of the unitary

        U|y⟩ = |xy mod n⟩  0 <= y < n
        U|y⟩ = |y⟩         n <= y

    then computes r (by continued fractions) if possible, and returns it.

    Args:
        result: result obtained by sampling the output of the
            circuit built by make_order_finding_circuit

    Returns:
        r, the order of x modulo n or None.
    """
    # Read the output integer of the exponent register.
    exponent_as_integer = result.data["exponent"][0]
    exponent_num_bits = result.measurements["exponent"].shape[1]
    eigenphase = float(exponent_as_integer / 2**exponent_num_bits)

    # Run the continued fractions algorithm to determine f = s / r.
    f = fractions.Fraction.from_float(eigenphase).limit_denominator(n)

    # If the numerator is zero, the order finder failed.
    if f.numerator == 0:
        return None

    # Else, return the denominator if it is valid.
    r = f.denominator
    if x**r % n != 1:
        return None
    return r

In [ ]:
"""Example of the classical post-processing."""

# Set n and x here
n = 6
x = 5

print(f"Finding the order of x = {x} modulo n = {n}\n")
measurement = cirq.sample(circuit, repetitions=1)
print("Raw measurements:")
print(measurement)

print("\nInteger in exponent register:")
print(measurement.data)

r = process_measurement(measurement, x, n)
print("\nOrder r =", r)
if r is not None:
    print(f"x^r mod n = {x}^{r} mod {n} = {x**r % n}")

Finding the order of x = 5 modulo n = 6

Raw measurements:
exponent=1, 0, 0, 0, 0, 0, 0, 0, 0

Integer in exponent register:
   exponent
0       256

Order r = 2
x^r mod n = 5^2 mod 6 = 1


## Understanding the Quantum Order Finder

At this point, you've built all the major quantum components of **Shor's algorithm**:

1. **Modular Exponentiation Gate (`ModularExp`)** — Encodes the operation
   $U|y⟩ = |xy \mod n⟩$
2. **Quantum Circuit Construction (`make_order_finding_circuit`)** — Implements **Quantum Phase Estimation** on ( U )
3. **Classical Post-Processing (`process_measurement`)** — Extracts the **order ( r )** using continued fractions
4. **Final Quantum Order Finder (`quantum_order_finder`)** — Ties everything together

Here's the full function again for clarity:

```python
def quantum_order_finder(x: int, n: int) -> int | None:
    """Computes smallest positive r such that x**r mod n == 1.

    Args:
        x: integer whose order is to be computed, must be greater than one
           and belong to the multiplicative group of integers modulo n (which
           consists of positive integers relatively prime to n),
        n: modulus of the multiplicative group.
    """
    # Check that the integer x is a valid element of the multiplicative group
    # modulo n.
    if x < 2 or n <= x or math.gcd(x, n) > 1:
        raise ValueError(f'Invalid x={x} for modulus n={n}.')

    # Create the order finding circuit.
    circuit = make_order_finding_circuit(x, n)

    # Sample from the order finding circuit.
    measurement = cirq.sample(circuit)

    # Return the processed measurement result.
    return process_measurement(measurement, x, n)
```

### How It Works — Step by Step

1. **Input Validation**

   * The algorithm requires that $\gcd(x, n) = 1$ and $1 < x < n$.
   * Otherwise, $x$ does not belong to the multiplicative group $\mathbb{Z}_n^*$.

2. **Quantum Phase Estimation (QPE)**

   * `make_order_finding_circuit(x, n)` builds a circuit that estimates the eigenvalue of the modular multiplication operator $U$.
   * The measured phase corresponds approximately to $s / r$, where $r$ is the **order** we seek.

3. **Measurement and Post-processing**

   * `cirq.sample(circuit)` executes the quantum circuit to get classical bitstring results.
   * `process_measurement()` converts the measurement result into a rational approximation of $s / r$ using **continued fractions**, retrieves the **order $r$**, and validates it by checking $x^r \mod n = 1$.

4. **Output**

   * Returns $r$, the **order of $x$** modulo $n$, or `None` if the algorithm fails (as it sometimes does probabilistically).

### Example Run

You can now test the **quantum order finder** directly:

```python
n = 15
x = 7
r = quantum_order_finder(x, n)

print(f"\nThe order of x = {x} modulo n = {n} is r = {r}")
if r is not None:
    print(f"Check: {x}^{r} mod {n} = {x**r % n}")
```

**Expected Output:**
For ( n = 15 ) and ( x = 7 ):

* The order ( r = 4 ), since ( 7^4 \mod 15 = 1 ).

### Interpretation

The **quantum order finder** is the **core quantum subroutine** of **Shor's algorithm**.
It's the only part where quantum computation provides an exponential speedup over classical methods.

Once we have $r$, the rest of the algorithm (factoring $n$) is purely classical:

1. Compute $\gcd(x^{r/2} ± 1, n)$
2. These typically yield the **prime factors of $n$**

### Next Step — Full Shor's Algorithm

Now that we can find the order $r$, the next step is to build the **full Shor's algorithm** that:

* Picks a random $x$
* Uses `quantum_order_finder(x, n)` to get $r$
* Uses classical post-processing to recover the **non-trivial factors of $n$**


In [ ]:
def quantum_order_finder(x: int, n: int) -> int | None:
    """Computes smallest positive r such that x**r mod n == 1.

    Args:
        x: integer whose order is to be computed, must be greater than one
           and belong to the multiplicative group of integers modulo n (which
           consists of positive integers relatively prime to n),
        n: modulus of the multiplicative group.
    """
    # Check that the integer x is a valid element of the multiplicative group
    # modulo n.
    if x < 2 or n <= x or math.gcd(x, n) > 1:
        raise ValueError(f'Invalid x={x} for modulus n={n}.')

    # Create the order finding circuit.
    circuit = make_order_finding_circuit(x, n)

    # Sample from the order finding circuit.
    measurement = cirq.sample(circuit)

    # Return the processed measurement result.
    return process_measurement(measurement, x, n)

## The Complete Factoring Algorithm

We now have:

* **Quantum order finder (`quantum_order_finder`)** — the quantum core
* **Classical order finder (for testing)**
* **Post-processing to derive factors from the order**

Now we build the **full Shor’s algorithm**, implemented in `find_factor`, which:

1. Handles **special cases** that are easy to detect classically.
2. Uses **quantum order finding** to compute the order $r$.
3. Derives a **non-trivial factor** from $r$.

### Step 1 — Handling Easy Classical Cases

```python
def find_factor_of_prime_power(n: int) -> int | None:
    """Returns non-trivial factor of n if n is a prime power, else None."""
    for k in range(2, math.floor(math.log2(n)) + 1):
        c = math.pow(n, 1 / k)
        c1 = math.floor(c)
        if c1**k == n:
            return c1
        c2 = math.ceil(c)
        if c2**k == n:
            return c2
    return None

**Purpose:**
Detects if $n = p^k$ for some prime $p$.
If yes, $p$ is trivially a factor.

**Example:**

* $n = 27 = 3^3$ → returns 3
* $n = 16 = 2^4$ → returns 2

This saves quantum computation time by removing unnecessary cases.

### Step 2 — Full Factoring Routine

```python
def find_factor(
    n: int,
    order_finder: Callable[[int, int], int | None] = quantum_order_finder,
    max_attempts: int = 30,
) -> int | None:
    """Returns a non-trivial factor of composite integer n."""
```

#### 🔹 Step-by-step logic:

1. **Check for trivial cases**

   ```python
   if sympy.isprime(n):
       print("n is prime!")
       return None
   if n % 2 == 0:
       return 2
   ```

   If $n$ is prime or even, the answer is trivial.

2. **Check for prime power**

   ```python
   c = find_factor_of_prime_power(n)
   if c is not None:
       return c
   ```

3. **Main quantum loop (up to max_attempts times)**

   ```python
   for _ in range(max_attempts):
       x = random.randint(2, n - 1)
       c = math.gcd(x, n)
       if 1 < c < n:
           return c
   ```

   * Pick a random base $x$
   * If $\gcd(x, n) > 1$, we immediately found a factor!

4. **Compute the order $r$ using the order finder**

   ```python
   r = order_finder(x, n)
   if r is None:
       continue
   if r % 2 != 0:
       continue
   ```

   * If the order finder fails or $r$ is odd, retry.

5. **Compute potential factor using the order**

   ```python
   y = x ** (r // 2) % n
   assert 1 < y < n
   c = math.gcd(y - 1, n)
   if 1 < c < n:
       return c
   ```

   If ( r ) is even, then:
   $$
   y = x^{r/2} \mod n
   $$
   and one of:
   $$
   \gcd(y-1, n) \quad \text{or} \quad \gcd(y+1, n)
   $$
   gives a **non-trivial factor** of $n$.

6. **If all attempts fail**

   ```python
   print(f"Failed to find a non-trivial factor in {max_attempts} attempts.")
   return None
   ```

### Step 3 — Using the Classical Order Finder

Because simulating quantum circuits for large ( n ) is computationally heavy, the example replaces the quantum order finder with a **classical substitute** for testing:

```python
p = find_factor(n, order_finder=classical_order_finder)
```

This allows you to **verify the logic** of Shor's algorithm using classical tools.


### Example 1 — Factoring a Medium-Sized Number

```python
# Number to factor
n = 184573

# Attempt to find a factor using the classical order finder
p = find_factor(n, order_finder=classical_order_finder)
q = n // p

print("Factoring n = pq =", n)
print("p =", p)
print("q =", q)

# Check correctness
print("Verification:", p * q == n)
```

This will output something like:

```
Factoring n = pq = 184573
p = 367
q = 503
Verification: True
```

### Example 2 — Factoring a Larger Number

```python
n = 1937089751
p = find_factor(n, order_finder=classical_order_finder)
q = n // p

print("Factoring n = pq =", n)
print("p =", p)
print("q =", q)
```

We'll again see correct factorization, confirming that Shor's algorithm's structure works end-to-end.


In [ ]:
"""Functions for factoring from start to finish."""


def find_factor_of_prime_power(n: int) -> int | None:
    """Returns non-trivial factor of n if n is a prime power, else None."""
    for k in range(2, math.floor(math.log2(n)) + 1):
        c = math.pow(n, 1 / k)
        c1 = math.floor(c)
        if c1**k == n:
            return c1
        c2 = math.ceil(c)
        if c2**k == n:
            return c2
    return None


def find_factor(
    n: int,
    order_finder: Callable[[int, int], int | None] = quantum_order_finder,
    max_attempts: int = 30,
) -> int | None:
    """Returns a non-trivial factor of composite integer n.

    Args:
        n: Integer to factor.
        order_finder: Function for finding the order of elements of the
            multiplicative group of integers modulo n.
        max_attempts: number of random x's to try, also an upper limit
            on the number of order_finder invocations.

    Returns:
        Non-trivial factor of n or None if no such factor was found.
        Factor k of n is trivial if it is 1 or n.
    """
    # If the number is prime, there are no non-trivial factors.
    if sympy.isprime(n):
        print("n is prime!")
        return None

    # If the number is even, two is a non-trivial factor.
    if n % 2 == 0:
        return 2

    # If n is a prime power, we can find a non-trivial factor efficiently.
    c = find_factor_of_prime_power(n)
    if c is not None:
        return c

    for _ in range(max_attempts):
        # Choose a random number between 2 and n - 1.
        x = random.randint(2, n - 1)

        # Most likely x and n will be relatively prime.
        c = math.gcd(x, n)

        # If x and n are not relatively prime, we got lucky and found
        # a non-trivial factor.
        if 1 < c < n:
            return c

        # Compute the order r of x modulo n using the order finder.
        r = order_finder(x, n)

        # If the order finder failed, try again.
        if r is None:
            continue

        # If the order r is even, try again.
        if r % 2 != 0:
            continue

        # Compute the non-trivial factor.
        y = x ** (r // 2) % n
        assert 1 < y < n
        c = math.gcd(y - 1, n)
        if 1 < c < n:
            return c

    print(f"Failed to find a non-trivial factor in {max_attempts} attempts.")
    return None

In [ ]:
"""Example of factoring via Shor's algorithm (order finding)."""

# Number to factor
n = 15

# Attempt to find a factor
p = find_factor(n, order_finder=classical_order_finder)
q = n // p

print("Factoring n = pq =", n)
print("p =", p)
print("q =", q)

Factoring n = pq = 15
p = 3
q = 5


In [ ]:
"""Example of factoring via Shor's algorithm (order finding)."""

# Number to factor
n = 184573

# Attempt to find a factor
p = find_factor(n, order_finder=classical_order_finder)
q = n // p

print("Factoring n = pq =", n)
print("p =", p)
print("q =", q)

Factoring n = pq = 184573
p = 487
q = 379


In [ ]:
import time
import numpy as np
from cirq import depolarize, amplitude_damp

def quantum_order_finder_with_shots(x: int, n: int, shots: int = 100) -> dict:
    """Quantum order finder with multiple shots and statistics"""
    circuit = make_order_finding_circuit(x, n)

    # Get the classical order for comparison
    classical_r = classical_order_finder(x, n)

    # Run multiple shots
    measurements = cirq.sample(circuit, repetitions=shots)

    results = {
        'x': x,
        'n': n,
        'classical_r': classical_r,
        'shots': shots,
        'quantum_results': [],
        'success_count': 0
    }

    # Get the number of bits in the exponent register
    L = n.bit_length()
    exponent_num_bits = 2 * L + 3

    for i in range(shots):
        # Extract the measurement value for this shot
        measured_value = measurements.data.iloc[i]['exponent']

        # Process this measurement directly without creating Result objects
        eigenphase = float(measured_value / 2**exponent_num_bits)
        f = fractions.Fraction.from_float(eigenphase).limit_denominator(n)

        if f.numerator == 0:
            quantum_r = None
        else:
            quantum_r = f.denominator
            if x**quantum_r % n != 1:
                quantum_r = None

        is_success = quantum_r == classical_r

        results['quantum_results'].append({
            'shot': i,
            'measured_value': measured_value,
            'quantum_r': quantum_r,
            'success': is_success
        })

        if is_success:
            results['success_count'] += 1

    results['success_probability'] = results['success_count'] / shots
    return results

def add_depolarizing_noise(circuit, p=0.01):
    """Add depolarizing noise to the circuit"""
    noisy_circuit = cirq.Circuit()
    for moment in circuit:
        noisy_circuit.append(moment)
        for qubit in circuit.all_qubits():
            noisy_circuit.append(depolarize(p).on(qubit))
    return noisy_circuit

def analyze_circuit_resources(circuit, n):
    """Analyze quantum circuit resources"""
    L = n.bit_length()

    # Count different gate types
    gate_counts = {}
    for op in circuit.all_operations():
        gate_type = str(op.gate) if op.gate else "Unknown"
        gate_counts[gate_type] = gate_counts.get(gate_type, 0) + 1

    metrics = {
        'n': n,
        'L': L,
        'total_qubits': len(circuit.all_qubits()),
        'expected_qubits': 3*L + 3,
        'total_gates': len(list(circuit.all_operations())),
        'gate_counts': gate_counts,
        'circuit_depth': len(circuit),
    }

    return metrics

def run_resource_analysis(n_values):
    """Run resource analysis for different n values"""
    results = []
    for n in n_values:
        if n % 2 == 0:  # Skip even numbers for factoring
            continue

        # Get multiplicative group (handle case where n might be prime)
        try:
            group = multiplicative_group(n)
            if not group:
                continue
            x = random.choice(group)
        except:
            continue

        circuit = make_order_finding_circuit(x, n)
        metrics = analyze_circuit_resources(circuit, n)
        results.append(metrics)

        print(f"n={n}, L={metrics['L']}, qubits={metrics['total_qubits']}, "
              f"gates={metrics['total_gates']}, depth={metrics['circuit_depth']}")

    return results

def test_known_orders():
    """Test with known orders for small N"""
    test_cases = [
        (15, 2, 4),   # order of 2 mod 15 is 4
        (15, 4, 2),   # order of 4 mod 15 is 2
        (15, 7, 4),   # order of 7 mod 15 is 4
    ]

    print("Testing known orders:")
    print("n\tx\texpected_r\tquantum_r\tmatch")
    print("-" * 50)

    for n, x, expected_r in test_cases:
        # Classical verification
        classical_r = classical_order_finder(x, n)

        # Quantum computation (multiple shots)
        result = quantum_order_finder_with_shots(x, n, shots=10)

        # Find the most common valid quantum_r
        valid_results = [r['quantum_r'] for r in result['quantum_results'] if r['quantum_r'] is not None]
        if valid_results:
            most_common_r = max(set(valid_results),
                              key=lambda r: valid_results.count(r))
        else:
            most_common_r = None

        match = most_common_r == expected_r
        print(f"{n}\t{x}\t{expected_r}\t\t{most_common_r}\t\t{match}")

def analyze_success_probability(n, num_trials=5, shots_per_trial=20):
    """Analyze how success probability scales with different parameters"""
    success_rates = []

    for trial in range(num_trials):
        try:
            group = multiplicative_group(n)
            if not group:
                continue
            x = random.choice(group)
            result = quantum_order_finder_with_shots(x, n, shots_per_trial)
            success_rates.append(result['success_probability'])

            print(f"Trial {trial+1}: x={x}, success_rate={result['success_probability']:.3f}")
        except Exception as e:
            print(f"Trial {trial+1} failed: {e}")
            continue

    if success_rates:
        avg_success = np.mean(success_rates)
        std_success = np.std(success_rates)

        print(f"\nOverall: average_success={avg_success:.3f} ± {std_success:.3f}")
        return success_rates
    else:
        print("No successful trials")
        return []

def benchmark_factoring():
    """Benchmark factoring for different number sizes"""
    numbers_to_factor = [15, 21, 33, 35]

    results = []
    for n in numbers_to_factor:
        print(f"\nFactoring n={n}:")

        start_time = time.time()
        factor = find_factor(n, order_finder=classical_order_finder)
        end_time = time.time()

        if factor:
            result = {
                'n': n,
                'factor': factor,
                'time_seconds': end_time - start_time,
                'L': n.bit_length(),
                'qubits_required': 3 * n.bit_length() + 3
            }
            results.append(result)
            print(f"  Factor: {factor}, Time: {result['time_seconds']:.4f}s")
        else:
            print(f"  No factor found")

    return results

def main():
    """Run comprehensive analysis"""
    print("=== Shor's Algorithm Comprehensive Analysis ===\n")

    # 1. Test known orders
    test_known_orders()

    # 2. Resource analysis
    print("\n=== Resource Analysis ===")
    n_values = [15, 21, 35]
    resource_results = run_resource_analysis(n_values)

    # 3. Success probability analysis
    print("\n=== Success Probability Analysis ===")
    success_results = analyze_success_probability(15, num_trials=3, shots_per_trial=10)

    # 4. Benchmark factoring
    print("\n=== Factoring Benchmark ===")
    factoring_results = benchmark_factoring()

    return {
        'resource_analysis': resource_results,
        'success_analysis': success_results,
        'factoring_benchmark': factoring_results
    }

# Run the analysis
if __name__ == "__main__":
    results = main()

=== Shor's Algorithm Comprehensive Analysis ===

Testing known orders:
n	x	expected_r	quantum_r	match
--------------------------------------------------
15	2	4		4		True
15	4	2		2		True
15	7	4		4		True

=== Resource Analysis ===
n=15, L=4, qubits=15, gates=15, depth=4
n=21, L=5, qubits=18, gates=17, depth=4
n=35, L=6, qubits=21, gates=19, depth=4

=== Success Probability Analysis ===
Trial 1: x=2, success_rate=0.600
Trial 2: x=4, success_rate=0.800
Trial 3: x=14, success_rate=0.400

Overall: average_success=0.600 ± 0.163

=== Factoring Benchmark ===

Factoring n=15:
  Factor: 3, Time: 0.0000s

Factoring n=21:
  Factor: 7, Time: 0.0000s

Factoring n=33:
  Factor: 3, Time: 0.0000s

Factoring n=35:
  Factor: 7, Time: 0.0000s
